In [ ]:
"""
Author: 
Date: 25/12/02
"""

In [2]:
# 一般采用多路并行召回检索，vector retrievel + text retrievel (e.g. BM25 retriever, TFIDF related algo) -> top K
# RRF粗排 + reranker精排
# RRF: 本质上是将多路召回的结果按照排名映射到同一个值域空间，拉齐评分标准，作用类似机器学习里softmax函数

In [9]:
from typing import List

# It seems llama-index lib changes a lot and doc is messy.. change to use langchain instead..
import torch
# from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
# from llama_index.core.node_parser import SentenceSplitter
# from llama_index.core import StorageContext, load_index_from_storage
from langchain_community.embeddings import HuggingFaceEmbeddings
# from llama_index.llms.ollama import Ollama
# from llama_index.llms.huggingface import HuggingFaceLLM
# from llama_index.retrievers.bm25 import BM25Retriever
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
import jieba
import os
from dotenv import load_dotenv
import pandas as pd
import ast

In [2]:
QUESTION = '骨折了应该怎么办'

In [3]:
def nice_printter(ans):
    rows = []
    for doc in ans:
        payload = ast.literal_eval(doc.page_content.strip())
        rows.append({"source": doc.metadata["source"], **payload})
    df = pd.DataFrame(rows)
    display(df)

In [4]:
# documents = SimpleDirectoryReader("./data").load_data()
# documents[0]
loader = TextLoader('./data/medical_data.txt')
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 0,
    length_function = len,
    separators=['\n']
)
docs = text_splitter.split_documents(documents)
docs[0]

Document(metadata={'source': './data/medical_data.txt'}, page_content="{'question': '曲匹地尔片的用法用量', 'answer': '注意：同种药品可由于不同的包装规格有不同的用法或用量。本文只供参考。如果不确定，请参看药品随带的说明书或向医生询问。口服。一次50～100mg（1-2片），3次/日，或遵医嘱。'}")

In [13]:
embed_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-large-zh-v1.5" # "jinaai/jina-embeddings-v2-base-zh"
)
db = FAISS.from_documents(docs, embedding=embed_model)
# db.save_local('./storage/')

In [14]:
vector_ans = db.similarity_search(QUESTION, k=10)
nice_printter(vector_ans)

,source,question,answer
0,./data/medical_data.txt,骨折术后护理,骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复...
1,./data/medical_data.txt,肱骨外髁颈骨折吃什么好？,一、肱骨外髁颈骨折食疗方(下面资料仅供参考，详细需要咨询医生)方1 赤小豆适量煎服，加赤砂糖...
2,./data/medical_data.txt,手指骨折后弯不了怎么办,手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊...
3,./data/medical_data.txt,骨盆骨折的并发症是什么?,1.腹膜后血肿。骨盆各骨主要为松质骨，盆壁肌肉多，邻近又有许多动脉丛和静脉丛，血液供应丰富，...
4,./data/medical_data.txt,锁骨骨折多久能干活,骨折的愈合期一般在3个月左右，在骨折刚愈合时还是需要尽量避免进行负重工作，这时可以进行轻微的...
5,./data/medical_data.txt,脚趾骨折吃什么水果好,骨折是指骨结构的连续性完全或部分断裂。多见于儿童及老年人，中青年人也时有发生。病人常为—个部...
6,./data/medical_data.txt,胸骨骨折多久能恢复,如果胸骨骨折较轻微而且患者身体素质比较好，一般需要在一两个月左右可以恢复。但是如果严重的话一...
7,./data/medical_data.txt,盖氏骨折的症状是什么？,与损伤的严重程度呈正相关，移位不明显的骨折仅有疼痛、肿胀和压痛;移位明显者，桡骨将出现短缩、...
8,./data/medical_data.txt,骨折后吃什么水果最佳,骨折术后，都是吃瘦肉，蛋白质含量高的食物。比如纯奶、青虾、带鱼，都有利于骨折的生长，而且还要...
9,./data/medical_data.txt,脚扭骨折吃什么食物好,脚扭伤大多都是因为在剧烈的运动或者是负重持重的时候姿势不当，或者是不小心跌仆、牵拉和过度扭转...


In [15]:
def preprocessing_func(text: str) -> List[str]:
    return list(jieba.cut(text))
bm25 = BM25Retriever(docs=docs,k=10)
print(bm25.k)
bm25_retriever = bm25.from_documents(docs,preprocess_func=preprocessing_func)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/_7/k0c2nyfd70b5x3jcncjlfctr0000gn/T/jieba.cache


10


Loading model cost 0.219 seconds.
Prefix dict has been built successfully.


In [16]:
text_ans = bm25_retriever.invoke(QUESTION)
print('answer from vector text retriever:')
nice_printter(text_ans)

answer from vector text retriever:


,source,question,answer
0,./data/medical_data.txt,手指骨折后弯不了怎么办,手指骨骨折是指手部的骨折。较其他部位骨折多见，且多为多发性骨折，骨折后有典型的移位和畸形，诊...
1,./data/medical_data.txt,手腕骨折一直肿胀怎么办,手腕骨折一直肿胀被认为是血液循环不良的原因。建议首先注意休息，避免过度活动，尤其是手腕活动。...
2,./data/medical_data.txt,骨折后吃什么水果最佳,骨折术后，都是吃瘦肉，蛋白质含量高的食物。比如纯奶、青虾、带鱼，都有利于骨折的生长，而且还要...
3,./data/medical_data.txt,骨折术后护理,骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复...


In [23]:
vector_ans

[Document(id='be365947-a27a-4bb3-9643-11bf5f4acde0', metadata={'source': './data/medical_data.txt'}, page_content="\n{'question': '骨折术后护理', 'answer': '骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复后1到3天，肢体明显肿胀外固定太紧，要注意观察末梢血液循环的情况，及时调整绷带的松紧度一面太紧而引起肢体缺血坏死。骨折中后期因肿胀消退后，外固定可能会松动，所以必须定期复查，以免造成骨折断端移位。如果是有局部外伤引起了局部的剧烈疼痛畸形一般是有骨折了。外伤后应该首先充分休息制动，避免小腿再活动。尽快做片子检查。。如果拍片检查确定是骨折应该尽快局部外固定充分制动，避免骨折断端的活动引起出血水肿加重或引起血管神经的损伤。完全固定后消肿治疗。同时确定是否需要进一步手术固定治疗。住院环境和作息制度给病人制造一个安静、安全、舒适、卫生的住院环境，保持室内空气新鲜，帮助病人制定规律生活和作息制度，保证充足的睡眠。这也是骨折的术后护理措施。病情观察由于老年人体内器官，代偿能力差，机体易出现电解质紊乱、全身衰竭等并发症，所以老年骨折病人在手术时机体受到极大刺激后，尤其要注意生命体征及全身情况的观察，防止因骨折手术加重老年骨折病人原有的疾患。骨折的前期阶段。骨折患者这个时候骨折后刚经过治疗，骨折部位还是肿胀，有大片的淤血，经络不通，气血阻滞，这个时候的饮食应以活血化瘀，行气消散为主。饮食以清淡为主，如蔬菜、蛋类、水果、瘦肉等，忌食辛辣、燥热、油腻。这样有利于淤血的退消，脉络的通畅，比较有利于骨骼的愈合。有利于关节共能的恢复。最主要的阶段，这时患者症状基本都已经退化。骨骼开始愈合，饮食要以营养为主，以满足骨骼生长需要，促进骨骼的愈合。多吃些骨头汤，动物的肝脏，多补充一些维生素a、d、钙已及蛋白质。吃些青菜、包菜、萝卜等维生素c含量丰富的蔬菜，以促进骨痂生长和伤口愈合。'}"),
 Document(id='ba9851e1-3a20-4262-9bc9-3b1de6c75be3', metadata={'source': './data/medical_data.txt'}, page_conten

In [24]:
from langchain_core.documents import Document

def _doc_id(doc, fallback_idx):
    if isinstance(doc, Document):
        return doc.metadata.get("doc_id") or doc.metadata.get("source") or doc.page_content[:64]
    return doc or f"doc_{fallback_idx}"

def rrf(vector_results: List[str], text_results: List[str], k: int=10, m: int=60):
        """
        使用RRF算法对两组检索结果进行重排序
        
        params:
        vector_results (list): 向量召回的结果列表,每个元素是专利ID
        text_results (list): 文本召回的结果列表,每个元素是专利ID
        k(int): 排序后返回前k个
        m (int): 超参数
        
        return:
        重排序后的结果列表,每个元素是(文档ID, 融合分数)
        """
        
        doc_scores = {}
        for rank, doc in enumerate(vector_results):
            doc_id = _doc_id(doc, rank)
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)
        for rank, doc in enumerate(text_results):
            doc_id = _doc_id(doc, len(vector_results) + rank)
            doc_scores[doc_id] = doc_scores.get(doc_id, 0) + 1 / (rank + m)
        return sorted(doc_scores, key=doc_scores.get, reverse=True)[:k]

In [25]:
# combine text retriever and vector retriever
vector_results = [i.page_content for i in vector_ans]
text_results = [i for i in text_ans]
rrf_res = rrf(vector_results, text_results)
rrf_res

['./data/medical_data.txt',
 "\n{'question': '骨折术后护理', 'answer': '骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复后1到3天，肢体明显肿胀外固定太紧，要注意观察末梢血液循环的情况，及时调整绷带的松紧度一面太紧而引起肢体缺血坏死。骨折中后期因肿胀消退后，外固定可能会松动，所以必须定期复查，以免造成骨折断端移位。如果是有局部外伤引起了局部的剧烈疼痛畸形一般是有骨折了。外伤后应该首先充分休息制动，避免小腿再活动。尽快做片子检查。。如果拍片检查确定是骨折应该尽快局部外固定充分制动，避免骨折断端的活动引起出血水肿加重或引起血管神经的损伤。完全固定后消肿治疗。同时确定是否需要进一步手术固定治疗。住院环境和作息制度给病人制造一个安静、安全、舒适、卫生的住院环境，保持室内空气新鲜，帮助病人制定规律生活和作息制度，保证充足的睡眠。这也是骨折的术后护理措施。病情观察由于老年人体内器官，代偿能力差，机体易出现电解质紊乱、全身衰竭等并发症，所以老年骨折病人在手术时机体受到极大刺激后，尤其要注意生命体征及全身情况的观察，防止因骨折手术加重老年骨折病人原有的疾患。骨折的前期阶段。骨折患者这个时候骨折后刚经过治疗，骨折部位还是肿胀，有大片的淤血，经络不通，气血阻滞，这个时候的饮食应以活血化瘀，行气消散为主。饮食以清淡为主，如蔬菜、蛋类、水果、瘦肉等，忌食辛辣、燥热、油腻。这样有利于淤血的退消，脉络的通畅，比较有利于骨骼的愈合。有利于关节共能的恢复。最主要的阶段，这时患者症状基本都已经退化。骨骼开始愈合，饮食要以营养为主，以满足骨骼生长需要，促进骨骼的愈合。多吃些骨头汤，动物的肝脏，多补充一些维生素a、d、钙已及蛋白质。吃些青菜、包菜、萝卜等维生素c含量丰富的蔬菜，以促进骨痂生长和伤口愈合。'}",
 "{'question': '肱骨外髁颈骨折吃什么好？', 'answer': '一、肱骨外髁颈骨折食疗方(下面资料仅供参考，详细需要咨询医生)方1 赤小豆适量煎服，加赤砂糖少许温服之，本方适用于活血化瘀期。方2 猪骨头1000克，黄豆250克，加水小火烧烂，加盐姜调味分饮食之。方3 猪脊骨一具，洗净，红枣120克，莲子90克，降香、生甘草各9克，加水小火烧烂，加姜盐调味分多次饮之。方4 鲜

In [33]:
# rag result
prompt = '''
用户问题：
{}

检索出的文档：
{}
'''

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-0.5B-Instruct", device_map="auto", dtype=torch.bfloat16)
model = model.to(device)

messages = [
    {"role": "system", "content": "You are a friendly chatbot who always responds in the style of a pirate",},
    {"role": "user", "content": prompt.format(QUESTION, ''.join(rrf_res))}
]
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)

with torch.inference_mode():
    outputs = model.generate(tokenized_chat, max_new_tokens=2048) 
bl_ans = tokenizer.decode(outputs[0])
print(f'answer w rag: {bl_ans}')

answer w rag: <|im_start|>system
You are a friendly chatbot who always responds in the style of a pirate<|im_end|>
<|im_start|>user

用户问题：
骨折了应该怎么办

检索出的文档：
./data/medical_data.txt
{'question': '骨折术后护理', 'answer': '骨折病人术后护理要做好外固定夹板，石膏，支具的护理。要观察外固定的松紧度，骨折早期可能因整复后1到3天，肢体明显肿胀外固定太紧，要注意观察末梢血液循环的情况，及时调整绷带的松紧度一面太紧而引起肢体缺血坏死。骨折中后期因肿胀消退后，外固定可能会松动，所以必须定期复查，以免造成骨折断端移位。如果是有局部外伤引起了局部的剧烈疼痛畸形一般是有骨折了。外伤后应该首先充分休息制动，避免小腿再活动。尽快做片子检查。。如果拍片检查确定是骨折应该尽快局部外固定充分制动，避免骨折断端的活动引起出血水肿加重或引起血管神经的损伤。完全固定后消肿治疗。同时确定是否需要进一步手术固定治疗。住院环境和作息制度给病人制造一个安静、安全、舒适、卫生的住院环境，保持室内空气新鲜，帮助病人制定规律生活和作息制度，保证充足的睡眠。这也是骨折的术后护理措施。病情观察由于老年人体内器官，代偿能力差，机体易出现电解质紊乱、全身衰竭等并发症，所以老年骨折病人在手术时机体受到极大刺激后，尤其要注意生命体征及全身情况的观察，防止因骨折手术加重老年骨折病人原有的疾患。骨折的前期阶段。骨折患者这个时候骨折后刚经过治疗，骨折部位还是肿胀，有大片的淤血，经络不通，气血阻滞，这个时候的饮食应以活血化瘀，行气消散为主。饮食以清淡为主，如蔬菜、蛋类、水果、瘦肉等，忌食辛辣、燥热、油腻。这样有利于淤血的退消，脉络的通畅，比较有利于骨骼的愈合。有利于关节共能的恢复。最主要的阶段，这时患者症状基本都已经退化。骨骼开始愈合，饮食要以营养为主，以满足骨骼生长需要，促进骨骼的愈合。多吃些骨头汤，动物的肝脏，多补充一些维生素a、d、钙已及蛋白质。吃些青菜、包菜、萝卜等维生素c含量丰富的蔬菜，以促进骨痂生长和伤口愈合。'}{'question': '肱骨外髁颈骨折吃什么好？', 'answer': '一、肱骨外髁颈骨折食疗方

In [32]:
# baseline result
messages = [
    {"role": "system", "content": "You are a friendly chatbot who always responds in the style of a pirate",},
    {"role": "user", "content": f"用户问题：{QUESTION}"},
 ]
 
tokenized_chat = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(tokenized_chat, max_new_tokens=2048) 
bl_ans = tokenizer.decode(outputs[0])
print(f'answer w rag: {bl_ans}')

answer w rag: <|im_start|>system
You are a friendly chatbot who always responds in the style of a pirate<|im_end|>
<|im_start|>user
用户问题：骨折了应该怎么办<|im_end|>
<|im_start|>assistant
在遇到骨折的情况下，首要任务是确保安全。首先，立即停止任何可能加重伤势的活动，并保持冷静。使用绷带或衣物包扎伤口以防止进一步伤害。

如果情况允许，可以尝试移动受伤部位到一个舒适的地方休息，避免长时间站立或坐着。对于严重的情况，应尽快寻求医疗帮助。不要自行处理骨折，以免造成二次伤害或感染。

记住，正确的急救措施和及时就医都是预防骨折的重要步骤。希望您的朋友能够平安归来！<|im_end|>
